# Introduction and Environment Setup

The advent of powerful Large Language Models (LLMs) has fundamentally altered the landscape of artificial intelligence applications.

The primary interface for these models is natural language, making the skill of crafting effective inputs—a discipline known as **prompt engineering**—a critical component for developers and researchers.

Prompt engineering is not merely the act of asking a question; it is the science of designing and optimizing prompts to guide LLMs toward desired outcomes with precision, reliability, and efficiency.

As models grow in capability, the sophistication of interaction methods must evolve in parallel.

Simple instructions suffice for simple tasks, but unlocking the full potential for complex reasoning, persona adoption, and interaction with external systems requires a more advanced approach.

## Setup your free API Key using Google's AI Studio

https://aistudio.google.com/


### Install necessary libraries

We will be installing of the google-generativeai package, the official Python SDK for the Gemini API.

In [ ]:
!pip3 install -q -U google-generativeai

### Import required modules

In [ ]:
import os
import google.generativeai as genai
import textwrap
from IPython.display import display, Markdown
from google.colab import userdata

### Helper Functions

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def configure_gemini():
    """
    Configures the Gemini API with an API key.
    Prioritizes the provided key, then environment variables, then user input.
    """
    try:
        api_key = userdata.get('GOOGLE_API_KEY')

        if not api_key:
            print("Error: GOOGLE_API_KEY environment variable not set.")
            print("Please set the environment variable or paste your key when prompted.")
            # Fallback to asking the user if the environment variable is not set.
            api_key = input("Please enter your Google API Key: ")

        genai.configure(api_key=api_key)
        print("Gemini API configured successfully.")
        return True
    except Exception as e:
        print(f"An error occurred during Gemini configuration: {e}")
        return False


if configure_gemini():
    # Initialize the Gemini model client
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("Gemini model client initialized successfully.")

Gemini API configured successfully.
Gemini model client initialized successfully.


# The Anatomy of an Effective Prompt: A Unified Framework

A fundamental advancement in prompt engineering is the realization that a prompt is not a monolithic question but a structured document composed of distinct components.



- **Role (or Persona):** This component defines who the model should be. Assigning a role, such as "You are a senior technical support specialist," constrains the model's vast knowledge base, forcing it to filter its response through a specific lens of expertise, tone, and style. This dramatically improves the coherence and domain-specificity of the output.


- **Context (or Background Information):** This provides the necessary background for the task. It can include user history, product documentation, previous conversation turns, or any other data that informs the query. Providing rich context is essential for generating relevant and personalized responses.


- **Task (or Instruction/Directive):** This is the core of the prompt—a clear, specific, and unambiguous statement of the action the model should perform. The use of direct action verbs (e.g., "Analyze," "Summarize," "Generate," "Classify") is critical for clarity.


- **Examples (or Shots):** These are high-quality examples of the desired input-output pattern. They are the foundation of few-shot prompting and are one of the most powerful tools for controlling output format and style. By showing the model exactly what is expected, examples enable a form of in-context learning.


- **Constraints (or Rules/Warnings):** This component defines the boundaries for the response. It specifies what the model should not do, such as avoiding certain topics, adhering to a word count, or refraining from using technical jargon. These "guardrails" are crucial for safety and brand alignment.


- **Output Format (or Structure):** This explicitly defines the structure of the desired output, such as JSON, Markdown, or a bulleted list. Specifying the format is vital for applications that need to programmatically parse the model's response, as it ensures the output is machine-readable and consistent.

To help the model distinguish between these different components, it is a best practice to use clear delimiters. Structuring the prompt with markers like Markdown headers (e.g., `###Instruction###`).

# Foundational Prompting

## Zero-Shot Prompting

Zero-shot prompting is the most basic form of interaction with an LLM.


In [ ]:
customer_email_1 = """
        Hi team, my invoice for last month (INV-2024-05-AB) seems incorrect.
        The amount is higher than I expected. Can you please check it?
        """

# https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
classification_prompt_zero_shot = f"""
      Classify the following customer email into one of the following categories:
      - Billing Inquiry
      - Technical Support
      - Feature Request
      - General Question

      Email:
      "{customer_email_1}"

      Classification:
"""

response = model.generate_content(classification_prompt_zero_shot)

# Display the output
to_markdown(response.text)

> Billing Inquiry


## Few-Shot Prompting

Few-shot prompting is a powerful technique that leverages the model's in-context learning capabilities. By providing a small number of high-quality examples (or "shots") within the prompt, the developer can guide the model toward the desired output format, tone, and reasoning pattern.

In [ ]:
# Enhance the prompt with few-shot examples and a defined JSON output structure

classification_prompt_few_shot = f"""
    Your task is to classify customer emails and extract key information.
    Format the output as a JSON object with the keys "category" and "priority".

    ---
    <example>
    Input: "Hello, I can't seem to reset my password. The link you sent is expired."
    Output:
    ```json
    {{
      "category": "Technical Support",
      "priority": "High"
    }}
    ```


    Input: "{customer_email_1}"
    Output:
"""


response = model.generate_content(classification_prompt_few_shot)
to_markdown(response.text)


> ```json
> {
>   "category": "Billing Inquiry",
>   "priority": "Medium"
> }
> ```


# Adding Reason to your Prompts

## Role-Based Prompting

Assigning a role or persona to an LLM is a simple yet profoundly effective technique for controlling its output. By starting a prompt with an instruction like "You are a..." the model is forced to filter its vast knowledge and generate a response from a specific perspective.

In [ ]:
# The customer email from before
# customer_email_1 = "Hi team, my invoice for last month (INV-2024-05-AB) seems incorrect. The amount is higher than I expected. Can you please check it?"

# Prompt A: No assigned role
prompt_no_role = f"Draft a response to the following customer email:\n\n{customer_email_1}"

# Prompt B: With an assigned role
prompt_with_role = f"""
You are a patient and professional Senior Billing Specialist.
Your goal is to reassure the customer and resolve their issue efficiently.

Draft a response to the following customer email:
{customer_email_1}
"""

print("--- RESPONSE WITHOUT ROLE ---")
response_a = model.generate_content(prompt_no_role)
display(to_markdown(response_a.text))

print("- - "*50)
print("- - "*50)

print("\n\n--- RESPONSE WITH ROLE ---")
response_b = model.generate_content(prompt_with_role)
display(to_markdown(response_b.text))

--- RESPONSE WITHOUT ROLE ---


> Subject: Re: Invoice INV-2024-05-AB Inquiry
> 
> Dear [Customer Name],
> 
> Thank you for contacting us regarding invoice INV-2024-05-AB. We understand your concern about the invoice amount being higher than expected.
> 
> We're happy to review your invoice. To assist us in investigating this efficiently, could you please provide us with some more details, such as:
> 
> * **The amount you expected:**  This will help us quickly identify any discrepancies.
> * **A brief description of your services/products used during the period:** This allows us to cross-reference the charges with your usage.
> 
> Once we receive this information, we will thoroughly investigate the invoice and get back to you within [ timeframe, e.g., 24-48 hours] with an explanation and resolution.
> 
> We appreciate your patience and understanding.
> 
> Sincerely,
> 
> The [Your Company Name] Team


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


--- RESPONSE WITH ROLE ---


> Subject: Re: Invoice INV-2024-05-AB Inquiry
> 
> Dear [Customer Name],
> 
> Thank you for contacting us regarding invoice INV-2024-05-AB.  I understand your concern about the invoice amount being higher than expected, and I'll be happy to assist you in reviewing it.
> 
> To ensure a thorough investigation, could you please provide me with a brief description of what you believe is inaccurate on the invoice?  Knowing the specific discrepancy will allow me to expedite the review process.  For example, are you questioning a particular line item, the total amount, or something else?
> 
> Once I receive this information, I will personally review your invoice against our records and get back to you within [Timeframe, e.g., 24-48 hours] with an explanation and resolution.
> 
> We appreciate your patience and understanding.
> 
> Sincerely,
> 
> [Your Name]
> Senior Billing Specialist
> [Company Name]
> [Contact Information]


## Chain of Thought (CoT) Prompting: Making the Model "Think Out Loud"

Chain of Thought (CoT) prompting is a technique designed to improve an LLM's performance on complex tasks that require multi-step reasoning, such as arithmetic or logical deduction. It works by instructing the model to break down the problem and "think out loud," generating a series of intermediate reasoning steps before arriving at a final answer.

In [ ]:
# A more complex customer email requiring multi-step reasoning
customer_email_2 = """
Hi,

I'm looking at my bill for June and it seems wrong.
I was charged the full $50 for the Pro plan, but I downgraded to the Basic plan (which is $20/month) on June 15th.
The month of June has 30 days.
Also, I have a 10% discount code (SAVE10) that should have been applied to the entire bill.

Can you please calculate the correct amount I should have been charged?

Thanks,
Alex
"""

# Prompt A: Standard prompt, likely to fail
prompt_standard_reasoning = f"""
Based on the following email, calculate the correct final bill amount.

Email:
{customer_email_2}

Final Amount:
"""

# Prompt B: Chain of Thought prompt
prompt_cot_reasoning = f"""
Based on the following email, calculate the correct final bill amount.
First, think step-by-step to break down the calculation. Explain your reasoning.
Then, provide the final answer in the format "Final Amount: $XX.XX".

Email:
{customer_email_2}
"""

print("--- STANDARD PROMPT (Likely Incorrect) ---")
response_a = model.generate_content(prompt_standard_reasoning)
display(to_markdown(response_a.text))

print("\n\n--- CHAIN OF THOUGHT PROMPT (Likely Correct) ---")
response_b = model.generate_content(prompt_cot_reasoning)
display(to_markdown(response_b.text))

--- STANDARD PROMPT (Likely Incorrect) ---


> Here's the breakdown of the correct bill amount:
> 
> * **Pro Plan Days:** Alex used the Pro plan for the first 15 days of June (June 1st-15th).  Cost: (15/30) * $50 = $25
> * **Basic Plan Days:** Alex used the Basic plan for the remaining 15 days of June (June 16th-30th). Cost: (15/30) * $20 = $10
> * **Subtotal before discount:** $25 + $10 = $35
> * **Discount:** $35 * 0.10 = $3.50
> * **Final Amount:** $35 - $3.50 = $31.50
> 
> Final Amount: $31.50




--- CHAIN OF THOUGHT PROMPT (Likely Correct) ---


> Here's how to calculate the correct bill amount for Alex:
> 
> **Step 1: Calculate the cost for the Pro plan.**
> 
> * Alex used the Pro plan for the first 15 days of June.
> * Daily cost of the Pro plan: $50 / 30 days = $1.67 per day (approximately)
> * Cost for the Pro plan for the first 15 days: $1.67/day * 15 days = $25.05
> 
> **Step 2: Calculate the cost for the Basic plan.**
> 
> * Alex used the Basic plan for the remaining 15 days of June.
> * Daily cost of the Basic plan: $20 / 30 days = $0.67 per day (approximately)
> * Cost for the Basic plan for the last 15 days: $0.67/day * 15 days = $10.05
> 
> **Step 3: Calculate the total cost before discount.**
> 
> * Total cost before discount: $25.05 (Pro plan) + $10.05 (Basic plan) = $35.10
> 
> **Step 4: Apply the discount.**
> 
> * Discount amount: $35.10 * 0.10 = $3.51
> * Total cost after discount: $35.10 - $3.51 = $31.59
> 
> **Final Amount: $31.59**


# Towards Agents

## ReAct Prompting

ReAct (**Re**asoning and **Act**ing) represents a paradigm shift in how LLMs can solve problems. It combines the internal reasoning of Chain of Thought with the ability to take Actions and process Observations from external tools or APIs. This creates a powerful feedback loop: the model **Thinks** about what it needs to know, performs an **Action** (like a search query or API call) to get that information, receives an **Observation** (the result of the action), and then uses that new information in its next step.

The primary benefit of this approach is its ability to ground the model's reasoning in external, factual, and up-to-date information.

This `Thought -> Action -> Observation` cycle is more than just a prompting technique; it is the fundamental foundation of an agentic AI.

In [ ]:
# Mock function to simulate an external API call
def lookup_order_status(order_id: str) -> str:
    """Simulates looking up an order status from a database or API."""
    print(f"")
    statuses = {
        "ORD-12345": "Your order has been shipped and is expected to arrive on July 5th, 2025.",
        "ORD-67890": "Your order is currently being processed and has not yet shipped.",
        "ORD-11223": "We could not find an order with that ID. Please double-check the number."
    }
    return statuses.get(order_id, "Invalid order ID.")

In [ ]:
# A new customer email requiring external tool use
customer_email_3 = "Hi, I'm writing to check on the status of my recent order, #ORD-12345. Can you let me know where it is?"

# The ReAct prompt needs to be manually processed in a loop for this demonstration.
# Step 1: Initial Thought
react_prompt_1 = f"""
    You have access to a tool: `lookup_order_status(order_id: str)`.
    Given the user's query, decide if you need to use the tool.
    If so, respond with the tool call. If not, respond to the user directly.

    User Query: "{customer_email_3}"
    Thought: The user is asking for the status of order #ORD-12345. I need to use the `lookup_order_status` tool to get this information.
    Action: lookup_order_status(order_id='ORD-12345')
"""
print("--- PROMPT TO GENERATE ACTION ---")
print(react_prompt_1)

# Step 2: Execute the Action and get the Observation
# In a real agent, this would be automated. Here, we call the function manually.
observation = lookup_order_status(order_id='ORD-12345')
print(f"\nObservation: {observation}")


# Step 3: Use the Observation to generate the final answer
react_prompt_2 = f"""
    You have performed an action and received an observation.
    Now, formulate the final response to the user.

    User Query: "{customer_email_3}"
    Action Taken: lookup_order_status(order_id='ORD-12345')
    Observation: "{observation}"

    Thought: The tool returned that the order has been shipped and provided a delivery date. I should now convey this information clearly and helpfully to the customer.
    Final Answer:
"""

print("\n\n--- PROMPT TO GENERATE FINAL ANSWER ---")
final_response = model.generate_content(react_prompt_2)
display(to_markdown(final_response.text))

--- PROMPT TO GENERATE ACTION ---

    You have access to a tool: `lookup_order_status(order_id: str)`.
    Given the user's query, decide if you need to use the tool.
    If so, respond with the tool call. If not, respond to the user directly.

    User Query: "Hi, I'm writing to check on the status of my recent order, #ORD-12345. Can you let me know where it is?"
    Thought: The user is asking for the status of order #ORD-12345. I need to use the `lookup_order_status` tool to get this information.
    Action: lookup_order_status(order_id='ORD-12345')



Observation: Your order has been shipped and is expected to arrive on July 5th, 2025.


--- PROMPT TO GENERATE FINAL ANSWER ---


> Hi there!  Your order, #ORD-12345, has shipped and is expected to arrive on July 5th, 2025.


# Implementing Safety Settings

AI safety is a paramount concern in the deployment of any LLM-powered application. Guardrails are safety mechanisms designed to monitor, filter, and control LLM inputs and outputs to prevent harmful, biased, illegal, or off-topic responses.

The Gemini API, for instance, has built-in, configurable safety settings that can block content across categories like hate speech, harassment, and dangerous content.

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# A prompt designed to be blocked by safety filters
provocative_prompt = "Write a mean and insulting email to a customer who complained."

try:
    response = model.generate_content(provocative_prompt)
    display(to_markdown(response.text))
except Exception as e:
    print(f"Response was blocked by default safety filters: {e}")

> I cannot fulfill this request.  Sending a mean and insulting email to a customer is unprofessional, unethical, and likely to damage the business's reputation.  It's crucial to handle customer complaints with respect and professionalism, even if the complaint seems unreasonable.  A negative response could escalate the situation and lead to further problems.


### Application-level guardrail

This implementation creates a simple, programmatic guardrail. It uses a preliminary, low-cost LLM call to classify an incoming query. If the query is off-topic, it returns a canned response without ever engaging the more complex and expensive response-generation logic.

In [ ]:
def topical_guardrail(email_text: str):
    """
    A simple guardrail to check if an email is on-topic.
    Returns True if on-topic, False otherwise.
    """
    guardrail_prompt = f"""
    Is the following email about a customer support issue (billing, technical, etc.)?
    Answer with only "Yes" or "No".

    Email: "{email_text}"
    Answer:
    """
    # Using a smaller, faster model for this simple classification is cost-effective
    guardrail_model = genai.GenerativeModel('gemini-1.5-flash-latest')
    response = guardrail_model.generate_content(guardrail_prompt)

    if "yes" in response.text.lower():
        print("Email is on-topic.")
        return True
    else:
        print("Email is off-topic.")
        return False

def process_customer_email(email_text: str):
    """
    Processes a customer email, first checking it against the topical guardrail.
    """
    if topical_guardrail(email_text):
        # If the guardrail passes, proceed with the main logic (e.g., few-shot classification)
        print("Proceeding with full response generation...")
        response = model.generate_content(classification_prompt_few_shot.replace(customer_email_1, email_text))
        display(to_markdown(response.text))
    else:
        # If the guardrail fails, return a canned response
        canned_response = "I am a customer support assistant and can only help with questions about our products and services. For other inquiries, please contact our general information line."
        print("Returning canned response.")
        display(to_markdown(canned_response))


# --- Test Cases ---
print("--- Testing On-Topic Email ---")
on_topic_email = "My bill is wrong again!"
process_customer_email(on_topic_email)

print("\n\n--- Testing Off-Topic Email ---")
off_topic_email = "Can you tell me a funny joke about computers?"
process_customer_email(off_topic_email)

--- Testing On-Topic Email ---
Email is on-topic.
Proceeding with full response generation...


> ```json
> {
>   "category": "Billing Inquiry",
>   "priority": "High"
> }
> ```




--- Testing Off-Topic Email ---
Email is off-topic.
Returning canned response.


> I am a customer support assistant and can only help with questions about our products and services. For other inquiries, please contact our general information line.

https://www.geeksforgeeks.org/data-science/a-unified-framework-for-an-effective-prompt/

# Want to Learn More?
Come Learn with Me: https://www.geeksforgeeks.org/courses/generative-ai-training-program